# Trainers

> Implementing the training loop for CPU/GPU training in pytorch.

In [ ]:
#| default_exp trainers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from tqdm import tqdm
import torch
import numpy as np
from fastcore.all import *
from fastcore.utils import *
from loguru import logger
from fedai.utils import *
from fedai.metrics import *


# Trainer

The Trainer class implements the training loop with the `fit()` method. It accepts a client at initialization, which contains all necessary infromation 
to implement a training loop for a federated learning setup.

For modularity, we implement two trainers, a single device (cpu or gpu), and a distributed trainer (multi-gpu, single node).

### Building blocks of the Trainer

The basic building blocks for the trainer are : 
- `_run_batch`: A method that accepts a batch of data as input, and returns the logits and the loss resulting from the forward pass.
- `_run_epoch`: A method that iterate over the whole dataset, and calls the `_run_batch` function within its logic.
- `fit`: A function that run a loop over a `n_epochs` to excute multiple epochs. It calls the `_run_epoch` method every iteration.
 
  
The `fit` method returns a `history` object, which is a `dict` containing the summary statistics of both the train and test datasets performance in the current federated round.

In [ ]:
#| export
class Trainer:
    def __init__(self, client):
        self.client = client
        self.cfg = client.cfg
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        self.client.train_loader = prepare_dl(self.cfg, self.client.train_ds)  # noqa: F405
        self.client.test_loader = prepare_dl(self.cfg, self.client.test_ds) # noqa: F405

        self.training_metrics = Metrics(list(self.cfg.training_metrics))  # noqa: F405
        self.test_metrics = Metrics(list(self.cfg.test_metrics))  # noqa: F405

        self.data_key, self.label_key = 'x', 'y'

Since different model have different input shapes that are expicted in their forward method, we define the general case here and sub-class's specfic method in each child class.

In [ ]:
#| export
@patch
def get_batch(self: Trainer, batch):
    return {k: v.to(self.device) for k, v in batch.items()}

Also, since LLMs usually pass a diciotnary to the `forward` method, we need to make sure to make class-specfic forward. For most cases, the usual `outputs = model(input)` would be sufficent, but for LLMs, we will define a child method that accepts a dicitonary not a regular tensor.

In [ ]:
#| export
@patch
def _forward(self: Trainer, batch):
    X, y = batch['x'], batch['y']
    outputs = self.client.model(X)
    loss = self.client.criterion(outputs, y)
    return loss, outputs

In [ ]:
#| export
@patch
def _closure(self: Trainer, batch: dict) -> tuple:
    try:
        loss, logits = self._forward(batch)
        probs =  torch.nn.functional.softmax(logits, dim= -1)
        y_pred = probs.argmax(dim= -1)#.view(-1)
        y_true = batch[self.label_key]#.view(-1)

        if self.cfg.training_metrics:

            if hasattr(self.client, "tokenizer"):
                metrcis = self.training_metrics.compute(y_pred= y_pred, y_true= y_true, tokenizer= self.client.tokenizer)
            else:
                metrcis = self.training_metrics.compute(y_pred= y_pred, y_true= y_true)
                
        else:
            metrcis = {k: 0 for k in self.training_metrics}
            
    except Exception as e:
        # print(f"Error in loss calculation: {e}")
        metrcis = {k: 0 for k in self.training_metrics}
        return torch.tensor(float(0), device=self.device), metrcis
        
    return loss, metrcis

In [ ]:
#| export
@patch
def _run_batch(self: Trainer, batch: dict) -> tuple:
    loss, metrics = self._closure(batch)
    self.client.model.zero_grad(set_to_none=True)

    if loss.item() == 0:
        return loss, metrics
    
    loss.backward()
    if self.cfg.grad_norm_clip:
        torch.nn.utils.clip_grad_norm_(self.client.model.parameters(), self.cfg.grad_norm_clip)

    self.client.optimizer.step()

    return loss, metrics

In [ ]:
#| export
@patch
def _run_epoch(self: Trainer):
    total_loss = 0
    lst_metrics= [] 
    num_trained = 0
    progress_bar = tqdm(range(len(self.client.train_loader)))

    self.client.model.train()
    for i, batch in enumerate(self.client.train_loader):
            
        batch = self.get_batch(batch)
        loss, metrics = self._run_batch(batch)

        if num_trained == 0:
            num_trained = 1e-10

        # print(f'Batch loss is {loss}')
        # progress_bar.update(1)
        # progress_bar.set_description(f'client {self.client.id} total_loss at step {i}, loss: {total_loss / num_trained if num_trained != 0 else 0.0}')
        
        if loss.item() != 0:
            total_loss += loss.item()
            num_trained += len(batch[self.data_key])
            lst_metrics.append(metrics)

    # average the metrics of the epoch (across batches)
    epoch_metrics = {k: sum([m[k] for m in lst_metrics]) / len(lst_metrics) for k in lst_metrics[0].keys()}

    return total_loss / num_trained, epoch_metrics

In [ ]:
#| export
@patch
def test(self: Trainer) -> dict:
    total_loss = 0
    lst_metrics = []
    # print("****************************************")
    # print(f'Inside the test () function of client {self.client.id}')
    
    self.client.model = self.client.model.to(self.device)
    self.client.model.eval()
    num_eval = 0
    
    with torch.no_grad():
        for i, batch in enumerate(self.client.test_loader):
            
            batch = self.get_batch(batch)

            if num_eval == 0:
                num_eval = 1e-10

            loss, metrics = self._closure(batch)                 

            # print(f"Client {self.client.id}'s Batch loss inside eval() : {loss}")

            if (not torch.isnan(loss)) and (self.cfg.grad_norm_clip <= 0 or loss != 0.0):
                total_loss += loss.item()  
                num_eval += len(batch[self.data_key])
                lst_metrics.append(metrics)           
            
        # print(f'Client {self.client.id} Eval loss is : {total_loss / num_eval}')
        # print("****************************************")
    epoch_metrics = {k: sum([m[k] for m in lst_metrics]) / len(lst_metrics) for k in lst_metrics[0].keys()}

    return total_loss / num_eval, epoch_metrics

Similar to `Keras` approach to return a history object, we make the same approach by invking the `get_gistory` function which averages the local metrics (usually `loss` and `accuraccy`).

In [ ]:
#| export
@patch
def train(self: Trainer) -> dict:
    init_test_loss, metrics_test = self.test()
    test_loss = [init_test_loss]
    test_metrics = [metrics_test]
    
    train_loss = []
    train_metrics = []
    for _ in range(self.cfg.local_epochs):
        
        self.client.model = self.client.model.to(self.device)

        avg_train_loss, metrics_train = self._run_epoch()
        train_loss.append(avg_train_loss)
        train_metrics.append(metrics_train)
        
        avg_test_loss, metrics_test = self.test()
        test_loss.append(avg_test_loss)   
        test_metrics.append(metrics_test)

    # average the metrics across all local rounds
    train_metrics = {k: sum([m[k] for m in train_metrics]) / len(train_metrics) for k in train_metrics[0].keys()}
    test_metrics = {k: sum([m[k] for m in test_metrics]) / len(test_metrics) for k in test_metrics[0].keys()}

    # add train_ to any key in train_metrics
    train_metrics = {f'train_{k}': v for k, v in train_metrics.items()}
    test_metrics = {f'test_{k}': v for k, v in test_metrics.items()}

    all_metrics =  {
        'train_loss': np.mean(train_loss),
        'test_loss': np.mean(test_loss),
    }

    # add the train and test metrics to the losses
    all_metrics.update(train_metrics)
    all_metrics.update(test_metrics)
    
    return all_metrics
    


In [ ]:
a = {'1': 1, '2': 2}
a.update({'3': 3})
a.update({'4': 4})

## LLMTrainer

This is an LLM-specific trainer. To define the custom LLM trainer, you need to define two methods `get_batch` and `_forward`. Since LLM's specific way of getting the batches are different from the usual pytorch way, you need to define a way to extract the batches out of the data loaders.

In [ ]:
#| export
class LLMTrainer(Trainer):
    def __init__(
        self,
        client
    ) -> None:        
        super().__init__(client)
        
        self.client.train_iterator = iter(self.client.train_loader)
        self.client.model.generation_config.pad_token_id = self.client.tokenizer.pad_token_id
        self.data_key, self.label_key = 'input_ids', 'labels'

In [ ]:
#| export
@patch
def get_batch(self: LLMTrainer, batch):  # noqa: F811
    return {
                'input_ids': batch['input_ids'].to(self.device),
                'labels': batch['labels'].to(self.device),
                'attention_mask': batch['attention_mask'].to(self.device) 
            }
    

In [ ]:
#| export
@patch
def _forward(self: LLMTrainer, batch):
    outputs = self.client.model(**batch)
    loss = self.client.criterion(outputs)
    return loss, outputs.logits

### LLM-specifc text generation test

The following two methods computes the text generation capabilities of an LLM. They use the metrics defined in the config file of the expirement. Usually you would use something like `Belu` or `Rouge` here.

Another more accurate and reliable way of assessing the output of an LLM is to use a metric like the Rouge-l, which assses the quality of the generated text not the quality of the raw model ouptus `logits`. This assessment is only done after the federated training process is done.

In [ ]:
#| export
@patch
def test_generate(self: LLMTrainer) -> float:
    # print("****************************************")
    # print(f'Inside the test_generate () function of client {self.client.id}')

    lst_metrics = []
    self.client.model = self.client.model.to(self.device)
    self.client.model.eval()
    
    progress_bar_eval = tqdm(range(len(self.client.test_loader_genr)))

    with torch.no_grad():
        for batch in self.client.test_loader_genr:

            batch = self.get_batch(batch)

            output_ids = self.client.model.generate(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                max_new_tokens=self.cfg.max_new_tokens,
                num_beams=self.cfg.num_beams,
            )

            generated_ids = output_ids[:, len(batch['input_ids'][0]):] 
            metrics = self.test_metrics.compute(y_pred= generated_ids,
                                                y_true= batch['labels'],
                                                tokenizer= self.client.tokenizer)
    
            lst_metrics.append(metrics)

            # print(f"Client {self.client.id}'s Batch test metrics are : {metrics}")
            progress_bar_eval.update(1)

    # print("****************************************")
    epoch_metrics = {k: sum([m[k] for m in lst_metrics]) / len(lst_metrics) for k in lst_metrics[0].keys()}

    return epoch_metrics


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()